In [1]:
import pandas as pd
import os
import mysql.connector

In [2]:
# !brew install mysql
# !pip install mysql
# !pip install mysql-connector-python

## DOMAIN DATA

In [3]:
%%capture

df = pd.read_csv('domain_data.csv')
domain_df = pd.DataFrame(columns=df.columns)

for idx in df.index:
    for dom in eval(df.loc[idx, 'Domains']):
        domain_df = domain_df.append({'Name': df.loc[idx, 'Name'], 'Domains': dom.lower()}, ignore_index=True)

In [4]:
domain_df['Domains'].nunique()

60

In [5]:
def clean_domains_df(df):
    df['Domains'].replace('image processing', 'computer vision', inplace=True)
    df['Domains'].replace('machine learning & deep learning', 'deep learning', inplace=True)
    df['Domains'].replace('machine learning algorithms', 'machine learning', inplace=True)
    df['Domains'].replace('signals and systems', 'signal processing', inplace=True)
    df.loc[df['Domains'].str.contains('signal processing'), 'Domains'] = 'signal processing'
    df.loc[df['Domains'].str.contains('computer vision'), 'Domains'] = 'computer vision'
    df.loc[df['Domains'].str.contains('audio'), 'Domains'] = 'audio processing'

    return df

In [6]:
domain_df = clean_domains_df(domain_df)
domain_df['Domains'].nunique()

52

In [7]:
domain_df['Domains'].unique()

array(['computer vision', 'deep learning', 'signal processing',
       'reinforcement learning', 'markov decision process',
       'wireless communication', 'resource allocation',
       'machine learning', 'audio processing', 'auditory neuroscience',
       'biometrics & human computer interactions (hci)', 'vi',
       'process engineering', 'separation process',
       'mathematical biology', 'mathematical finance',
       'computer networks', 'network security',
       'distributions models and its applications',
       'statistics and finance', 'nlp', 'clinical data mining',
       'computational biology', 'complex system', 'collective behavior',
       'dynamical system', 'statistical physics in biology', 'vlsi',
       'hpc', 'semiconductor devices', 'iot', 'information retrieval',
       'data mining', 'pattern recognition', 'applied mathematics',
       'non-linear dynamics', 'healthcare', 'robotics', 'biomechanics',
       'finite element analysis', 'optimization', 'implant de

In [8]:
domain_df.drop_duplicates(ignore_index=True, inplace=True)
domain_df.shape

(65, 2)

## DOI DATA

In [9]:
# paper_folder_path = 'DBMS-Lab-main/publication_data'
# ct = 0
# ct1 = 0

# for prof_file in sorted(os.listdir(paper_folder_path)):

#     try:
#         if prof_file.split('.')[-1] != 'csv': continue
#         publication_df = pd.read_csv(os.path.join(paper_folder_path, prof_file))
        
#         for idx in publication_df.index:
#             date = publication_df.loc[idx, 'Publication date']
#             if isinstance(date, str):
#                 if int(date.split('/')[0]) < 2018:
#                     publication_df.drop(idx, axis='rows', inplace=True)
#                     ct+=1
#             ct1+=1
#         publication_df.to_csv(paper_folder_path + '/' + prof_file, index=False)
            
#     except:
#         print(prof_file + " Failed")
        
# print(ct)
# print(ct1)

In [10]:
paper_folder_path = 'DBMS-Lab-main/publication_data'

paper_col_count = {}
# ct = 0

for prof_file in sorted(os.listdir(paper_folder_path)):

    try:
        if prof_file.split('.')[-1] != 'csv': continue
        publication_df = pd.read_csv(os.path.join(paper_folder_path, prof_file))
#         ct += len(publication_df)
        
        for col in publication_df.columns:
            if col in paper_col_count:
                paper_col_count[col] += publication_df[col].notna().sum()
            else:
                paper_col_count[col] = 0
                
    
    except:
        print(prof_file + " Failed")
        
# print(f"Num of Papers: {ct}")
paper_col_count

{'Title': 935,
 'Authors': 842,
 'Publication date': 686,
 'Journal': 408,
 'Volume': 301,
 'Issue': 184,
 'Publisher': 536,
 'Description': 832,
 'Total citations': 935,
 'Scholar articles': 853,
 'Citation Count': 935,
 'Pages': 554,
 'Conference': 198,
 'Book': 35,
 'Inventors': 11,
 'Patent office': 11,
 'Patent number': 7,
 'Application number': 11,
 'Institution': 5,
 'Source': 18,
 'Report number': 0}

In [11]:
# Combines all into published_in column with publication_type column as well
publication_columns = ['Journal', 'Conference', 'Patent office', 'Book']

# Combines both into authors column, basically got rid of inventors columns by merging it
writer_columns = ['Authors', 'Inventors']

# These columns are dropped
drop_columns = ['Volume', 'Issue', 'Pages', 'Patent number', 'Application number', 
             'Institution', 'Source', 'Report number']

In [12]:
def reformat_publications_df(publication_df):
    
        ###----------------Drop excess columns----------------###
        drop_col = list(set(publication_df.columns) & set(drop_columns))
        publication_df.drop(drop_col, axis='columns', inplace=True)
        
        ###----------------Combine publication columns----------------###
        
        publication_df['Published_in'] = ''
        publication_df['Publication_type'] = ''
        
        # Created published_in column
        publication_cols = list(set(publication_df.columns) & set(publication_columns))
        for col in publication_cols:
            publication_df['Published_in'] += publication_df[col].fillna('')
        
        # Created publication_type column
        publication_df[publication_cols] = publication_df[publication_cols].notna() * publication_cols
        for col in publication_cols:
            publication_df['Publication_type'] += publication_df[col]
            
        publication_df.drop(publication_cols, axis='columns', inplace=True)
        
        ###----------------Combine authors columns----------------###
        
        publication_df['AUTHORS'] = ''
        
        author_cols = list(set(publication_df.columns) & set(writer_columns))
        for col in author_cols:
            publication_df['AUTHORS'] += publication_df[col].fillna('')
            
        publication_df.drop(author_cols, axis='columns', inplace=True)
        publication_df.rename(columns = {'AUTHORS':'Authors'}, inplace = True)
        
        ###----------------Publication date column----------------###
        
        def date2year(dt):
            if isinstance(dt, str): 
                return dt.split('/')[0]
            else:
                return 0
        
        publication_df['Publication date'] = publication_df['Publication date'].apply(lambda dt : date2year(dt))
        publication_df = publication_df.astype({'Publication date':int})
        publication_df.rename(columns = {'Publication date':'Publication year'}, inplace = True)
    
        ###----------------Yearwise Citation columns----------------###
        
        years = ['2018', '2019', '2020', '2021', '2022', '2023']
        for idx in publication_df.index:
            year_dict = eval(publication_df.loc[idx, 'Citation Count'])
            for year in years:
                if year in year_dict:
                    publication_df.loc[idx, year + '_citations'] = year_dict[year]
                else:
                    publication_df.loc[idx, year + '_citations'] = 0
                    
        publication_df.drop('Citation Count', axis='columns', inplace=True)
        for year in years:
            publication_df = publication_df.astype({year+'_citations':int})
        
        ###----------------Fixing column names----------------###
        
        for col in publication_df.columns:
            if ' ' in col:
                new_name = '_'.join(col.split(' '))
                publication_df.rename(columns = {col:new_name}, inplace = True)
        
        return publication_df

In [13]:
pub_sql_columns = []

for prof_file in sorted(os.listdir(paper_folder_path)):

    try:
        if prof_file.split('.')[-1] != 'csv':
            continue
            
        publication_df = pd.read_csv(os.path.join(paper_folder_path, prof_file))
        
        publication_df = reformat_publications_df(publication_df)
        pub_sql_columns.extend(publication_df.columns)
        
        break
        
    except:
        print(prof_file + " Failed")
                
pub_sql_columns = list(set(pub_sql_columns))

publication_df

,Title,Publication_year,Publisher,Description,Total_citations,Scholar_articles,Published_in,Publication_type,Authors,2018_citations,2019_citations,2020_citations,2021_citations,2022_citations,2023_citations
0,Confidence Interval Construction for Multivari...,2022,NaN,In this paper we propose a novel procedure to ...,0,Confidence Interval Construction for Multivari...,arXiv preprint arXiv:2211.13915,Journal,"Aryan Bhambu, Arabin Kumar Dey",0,0,0,0,0,0
1,Integrated Brier Score based Survival Cobra--A...,2022,NaN,"In this paper, we provide two novel regression...",0,Integrated Brier Score based Survival Cobra--A...,arXiv preprint arXiv:2210.12006,Journal,"Rahul Goswami, Arabin Kumar Dey",0,0,0,0,0,0
2,Controlling Travel Path of Original Cobra,2022,NaN,In this paper we propose a kernel based COBRA ...,0,Controlling Travel Path of Original Cobra\nMB ...,arXiv preprint arXiv:2210.10655,Journal,"Mriganka Basu RoyChowdhury, Arabin K Dey",0,0,0,0,0,0
3,Controlling Travel Path of Original Cobra,2022,NaN,In this paper we propose a kernel based COBRA ...,0,Controlling Travel Path of Original Cobra\nM B...,arXiv e-prints,Journal,"Mriganka Basu RoyChowdhury, Arabin K Dey",0,0,0,0,0,0
4,Concordance based Survival Cobra with regressi...,2022,NaN,"In this paper, we predict conditional survival...",0,Concordance based Survival Cobra with regressi...,arXiv preprint arXiv:2209.11919,Journal,"Rahul Goswami, Arabin Kumar Dey",0,0,0,0,0,0
5,Modeling long-term groundwater levels by explo...,2021,Springer Netherlands,Inevitable issues concerning the sustainabilit...,10,Modeling long-term groundwater levels by explo...,Water Resources Management,Journal,"Sangita Dey, Arabin Kumar Dey, Rajesh Kumar Mall",0,0,0,0,5,5
6,Construction of confidence interval for a univ...,2020,Springer Berlin Heidelberg,"In this paper, we show an innovative way to co...",5,Construction of confidence interval for a univ...,Annals of Data Science,Journal,"Shankhajyoti De, Arabin Kumar Dey, Deepak Kuma...",0,0,0,1,3,1
7,Some Variations of EM Algorithms for Marshall–...,2019,Springer International Publishing,"Recently, Asimit et al. have used an EM algori...",3,Some Variations of EM Algorithms for Marshall–...,Journal of Statistical Theory and Practice,Journal,"Arabin Kumar Dey, Biplab Paul",3,0,0,0,0,0
8,Fate of Snakes in an Urban Landscape-A report ...,2019,NaN,NaN,6,Fate of Snakes in an Urban Landscape-A report ...,Reptile Rap,Journal,"D Gayen, S Dey, AK Dey, US Roy",0,0,0,0,4,1
9,Fate of Snakes in an Urban Landscape-A,2019,NaN,Materials and Methods\nStudy Site: The present...,0,Fate of Snakes in an Urban Landscape-A\nD Gaye...,,,"D Gayen, S Dey, AK Dey, US Roy",0,0,0,0,0,0


## SQL Connection

In [14]:
mydb = mysql.connector.connect(
  host="localhost",
  user="research_db_admin",
  password="1234",
  database="ResearchPortalDB"
)

db_cursor = mydb.cursor()

### Profs

In [15]:
prof_folder_path = 'DBMS-Lab-main/prof_data'
prof_data = pd.DataFrame(columns = ['Name', 'citations_all', 'citations_after2018', 'i10_index_all', 'i10_index_after2018', 'h_index_all', 'h_index_after2018', 'position', 'department', 'position_in_DSAI'])
prof_text_cols = ['Name', 'position', 'department', 'position_in_DSAI']

prof_pos_df = pd.read_csv('faculty.csv')
prof_pos_df['Position in DSAI'] = prof_pos_df['Position in DSAI'].str.replace('head', 'head of the school')
prof_pos_df['Position in DSAI'] = prof_pos_df['Position in DSAI'].str.replace('assoc_faculty_list', 'associated faculty')
prof_pos_df['Position in DSAI'] = prof_pos_df['Position in DSAI'].str.replace('distinguished_faculty', 'distinguished faculty')

###----------------Making Prof_data DataFrame----------------###
idx = 0 
for prof in sorted(os.listdir(prof_folder_path)):
    
    if prof.split('.')[-1] != 'csv':
        continue

    if prof == 'domain_data.csv':
        continue

    prof_df = pd.read_csv(os.path.join(prof_folder_path, prof)).set_index('Index')
    name = ' '.join(prof.replace('.csv', '').split('_'))
    
    prof_data.loc[idx, 'Name'] = name
    prof_data.loc[idx, 'citations_all'] = prof_df.loc['Citations', 'All']
    prof_data.loc[idx, 'citations_after2018'] = prof_df.loc['Citations', 'Since 2018']
    prof_data.loc[idx, 'i10_index_all'] = prof_df.loc['i10-index', 'All']
    prof_data.loc[idx, 'i10_index_after2018'] = prof_df.loc['i10-index', 'Since 2018']
    prof_data.loc[idx, 'h_index_all'] = prof_df.loc['h-index', 'All']
    prof_data.loc[idx, 'h_index_after2018'] = prof_df.loc['h-index', 'Since 2018']
    prof_data.loc[idx, 'position'] = prof_pos_df.loc[prof_pos_df['Name'] == name, 'Position'].squeeze().split(', ')[0]
    prof_data.loc[idx, 'department'] = prof_pos_df.loc[prof_pos_df['Name'] == name, 'Position'].squeeze().split(', ')[1]
    prof_data.loc[idx, 'position_in_DSAI'] = prof_pos_df.loc[prof_pos_df['Name'] == name, 'Position in DSAI'].squeeze()
   
    idx += 1
    
    
###----------------Creating Professors SQL Table----------------###

column_cmd = ""
for col in prof_data.columns:
    if col in prof_text_cols:
        column_cmd += f"{col} text, "
    else:
        column_cmd += f"{col} int, "
    
publications_table_command = f'CREATE TABLE professors (ProfID int NOT NULL AUTO_INCREMENT, PRIMARY KEY (ProfID), {column_cmd});'.replace(', )', ')')
db_cursor.execute(publications_table_command)
db_cursor.execute('ALTER TABLE professors AUTO_INCREMENT=2100;')

###----------------Uploading to Professors SQL Table----------------###

for idx in prof_data.index:
    
    col_names = ""
    values = ""
    for col in prof_data.columns:
        col_names += f"{col}, "
        values += f"\"{prof_data.loc[idx, col]}\", "
        
    upload_command = f"INSERT INTO professors({col_names}) VALUES ({values});".replace(', )', ')')
    db_cursor.execute(upload_command)
    mydb.commit()

In [16]:
integer_columns = ['Publication_year', '2018_citations', '2019_citations', '2020_citations', '2021_citations', '2022_citations', '2023_citations', 'Total_citations']

def create_publication_sql(pub_sql_columns, integer_columns):
    
    column_cmd = ""
    for col in pub_sql_columns:
        if col  not in integer_columns:
            column_cmd += f"{col} text, "
    
    for col in integer_columns:
        column_cmd += f"{col} int, "
        
    publications_table_command = f'CREATE TABLE publications (PaperID int NOT NULL AUTO_INCREMENT, PRIMARY KEY (PaperID), {column_cmd});'.replace(', )', ')')

    db_cursor.execute(publications_table_command)
    db_cursor.execute('ALTER TABLE publications AUTO_INCREMENT=11000;')
    
def create_author_relation_sql():

    auth_reln_command = "CREATE TABLE author_relations (ProfID int, PaperID int)"
    db_cursor.execute(auth_reln_command)
    
def create_coauthor_sql():

    coauth_command = "CREATE TABLE coauthors (PaperID int, Author text)"
    db_cursor.execute(coauth_command)
    
create_publication_sql(pub_sql_columns, integer_columns)
create_author_relation_sql()
create_coauthor_sql()

In [17]:
def update_publication_df(prof_file, df):
    
#     try:
        col_names = ''
        for col in df.columns:
            col_names += f'{col}, '
            
        # ProfID Querying    
        prof_name = ' '.join(prof_file.replace('.csv', '').split('_'))
        prof_id_query = f"SELECT ProfID FROM professors WHERE (Name = \"{prof_name}\")"
        db_cursor.execute(prof_id_query)
        prof_id = db_cursor.fetchall()[0][0]
        
        for idx in df.index:
            
            ## Create query and update command for author relation table and publication table
            values = ''
            query_condition = ''
            for col in df.columns:
                
                if(isinstance(df.loc[idx, col], str)):
                    df.loc[idx, col] = df.loc[idx, col].replace('"', "'")
                    df.loc[idx, col] = df.loc[idx, col].replace("\\", "")
                    df.loc[idx, col] = df.loc[idx, col].replace("\n", " ")
                    
                query_condition += f" ({col} = \"{df.loc[idx, col]}\") AND"
                values += f"\"{df.loc[idx, col]}\", "
            
            record_query = f"SELECT PaperID FROM publications WHERE ({query_condition})".replace('AND)', ')')
            update_command = f"INSERT INTO publications({col_names}) VALUES ({values})".replace(', )', ')')
            
            db_cursor.execute(record_query)
            query_result = db_cursor.fetchall()
            if len(query_result) == 0:
                db_cursor.execute(update_command)
                mydb.commit()
                
                db_cursor.execute("SELECT MAX(PaperID) FROM publications")
                paper_id = db_cursor.fetchall()[0][0] + 1

            else:
                paper_id = query_result[0][0]
                
            db_cursor.execute(f"INSERT INTO author_relations(ProfID, PaperID) VALUES ({prof_id}, {paper_id})")
            mydb.commit()
            
            ## Update coauthor table
            
            auths = df.loc[idx, 'Authors'].split(', ')
            for auth in auths:
                coauth_cmd = f"INSERT INTO coauthors(PaperID, Author) VALUES ({paper_id}, \"{auth}\");"
                db_cursor.execute(coauth_cmd)
                mydb.commit()
                
#     except:
#         print(str(df.loc[idx, 'Title']) + "  " + prof_file)
#         print(update_command)
#         print()
#         print()

In [18]:
titles = []
all_df = pd.DataFrame()
for prof_file in sorted(os.listdir(paper_folder_path)):

#     try:
        if prof_file.split('.')[-1] != 'csv':
            continue
            
        publication_df = pd.read_csv(os.path.join(paper_folder_path, prof_file))
        publication_df = reformat_publications_df(publication_df)
        update_publication_df(prof_file, publication_df)
        
#         break
        
#     except:
#         print(prof_file + " Failed")

In [19]:
# all_df[all_df['Title'] == 'Automatic syllabification for manipuri language']
# # all_df[all_df['Title'] == 'Comparison of floating-point representations for the efficient implementation of machine learning algorithms']
# len(all_df[['Title', 'Authors', 'Published_in']].value_counts().keys())

## Domain Table

In [20]:
def create_domain_table_sql():

    domain_table_command = "CREATE TABLE domains (ProfID int, Domains text)"
    db_cursor.execute(domain_table_command)
    
    for idx in domain_df.index:
        try:
            prof_name = domain_df.loc[idx, 'Name']
            prof_id_query = f"SELECT ProfID FROM professors WHERE (Name = \"{prof_name}\")"
            db_cursor.execute(prof_id_query)
            prof_id = db_cursor.fetchall()[0][0]

            domain_update_command = f"INSERT INTO domains(ProfID, Domains) VALUES ({prof_id}, \"{domain_df.loc[idx, 'Domains']}\")"
            db_cursor.execute(domain_update_command)
            mydb.commit()
        except:
            print(prof_name + " Failed")
        
create_domain_table_sql()

## Co author